In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
trainFile=pd.read_csv("gender_age_train.csv")
testFile=pd.read_csv("gender_age_test.csv")

In [3]:
#print(trainFile.shape)#(74654,4)
trainFile[0:1]

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38


In [4]:
#print(testFile.shape)#(112071,1)
testFile[0:1]

,device_id
0,1002079943728939269


In [5]:
#Understanding Target Value Characteristics(Regression w.r.t. Age, Classification w.r.t. Gender)
trainGrpBy=trainFile.groupby("group")
trainGroups=trainGrpBy.groups
trainGroupDict=[(key,len(trainGroups[key])) for key in trainGroups.keys()]
print(trainGroupDict)
#Female Groups are : F23-,F24-26,F27-28,F29-32,F33-42,F43+
#Male Groups are : M22-,M23-26,M27-28,M29-31,M32-38,M39+

[('F43+', 4194), ('M32-38', 9476), ('F23-', 5050), ('F24-26', 4190), ('M22-', 7488), ('M39+', 8581), ('F29-32', 4628), ('M27-28', 5445), ('F33-42', 5561), ('M29-31', 7309), ('F27-28', 3118), ('M23-26', 9605)]


In [6]:
len(trainFile.groupby('device_id').groups.keys())#Number of unique device_ids: 74645 i.e. All device_id are unique

74645

In [7]:
trainFile.describe()#Interesting we have a Kid of Age 1 and grandpa having age 96 using Mobile Device :-)
#But Most of the users are withing age group 25 to 36

,device_id,age
count,7.464500e+04,74645.000000
mean,-7.491354e+14,31.410342
std,5.327150e+18,9.868735
min,-9.223067e+18,1.000000
25%,-4.617367e+18,25.000000
50%,-1.841362e+16,29.000000
75%,4.636656e+18,36.000000
max,9.222849e+18,96.000000


In [8]:
eventsData=pd.read_csv("events_subset.csv")
print(eventsData[0:2])
#eventsData.describe()
#event_id is PK here;device_id may get duplicated
#longitude: -180 to 151 : Continuous
#latitude: -37 to 53.64 : Continuous

   event_id            device_id            timestamp  longitude  latitude
0         1    29182687948017175  2016-05-01 00:55:25     121.38     31.24
1         2 -6401643145415154744  2016-05-01 00:54:12     103.65     30.97


In [9]:
deviceIdGroup=eventsData.groupby("device_id").groups
deviceIDGroupDict=[(key,len(deviceIdGroup[key])) for key in deviceIdGroup.keys()]
#sorted(deviceIDGroupDict)
mostActiveDeviceIds=sorted(deviceIDGroupDict,key=lambda x: x[1],reverse=True)[0:10]
print(mostActiveDeviceIds)

[(1186608308763918427, 937), (3915082290673137129, 493), (-1656894751624916732, 229), (2504414082456157897, 135), (-3746248670824158209, 115), (-7369693784883843916, 113), (8339429008953975436, 110), (-6242501228649113250, 110), (-17299534936664237, 108), (-8340098378141155823, 104)]


In [20]:
#As expected => Very High Correlation between longitude and latitude : 0.947287
eventsData.corr()

,event_id,device_id,longitude,latitude
event_id,1.000000,0.001050,-0.003879,-0.004633
device_id,0.001050,1.000000,-0.013765,-0.007659
longitude,-0.003879,-0.013765,1.000000,0.947287
latitude,-0.004633,-0.007659,0.947287,1.000000


In [18]:
deviceIdGroupObj=eventsData.groupby("device_id")
deviceIdGroupObj.ngroups#27310 Unique values of Device_id
eventsData[eventsData.device_id==29182687948017175]#Observation Location Doesn't change that frequently
eventsDataFor1Device=eventsData[eventsData.device_id==mostActiveDeviceIds[0][0]]
eventsDataFor1Device=eventsDataFor1Device.sort_values(['timestamp'])
eventsDataFor1Device.plot(x='timestamp',y='longitude')
eventsDataFor1Device.plot(x='timestamp',y='latitude')
plt.show()#Very Random Movement in long and lat data : Seems like this Person travels a lot

In [22]:
eventsDataFor2Device=eventsData[eventsData.device_id==mostActiveDeviceIds[1][0]].sort_values(['timestamp'])
eventsDataFor2Device.plot(x='timestamp',y='longitude')
eventsDataFor2Device.plot(x='timestamp',y='latitude')
plt.show()#Again the second most active person also moves/travels a lot
#Simailar to 1st person=>long and lat values are frequently dropping to 0[Not sure if there is N/w issue at his place]

In [23]:
eventsDataFor3Device=eventsData[eventsData.device_id==mostActiveDeviceIds[2][0]].sort_values(['timestamp'])
eventsDataFor3Device.plot(x='timestamp',y='longitude')
eventsDataFor3Device.plot(x='timestamp',y='latitude')
plt.show()#If we ignore sudden 0's : 3rd most active person doesn't move/travel a lot
#Simailar to 1st and 2nd person=>long and lat values are frequently 0[N/w issue ??]

In [24]:
#eventsData[]
np.where(eventsData['device_id']==-9220329415676028483)

(array([ 4529, 35717], dtype=int32),)

In [25]:
app_eventsData=pd.read_csv("app_events_subset.csv")

In [26]:
app_eventsData[0:4]

,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1
1,2,-5720078949152207372,1,0
2,2,-1633887856876571208,1,0
3,2,-653184325010919369,1,1
